In [3]:
print("Notebook is working")

Notebook is working


In [37]:
import requests

#Weather API endpoint
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": date,
    "end_date": date, 
    "hourly": "wind_direction_10m,wind_speed_10m",
    "timezone": "America/New_York"
}

response = requests.get(url, params = params)
data = response.json()

In [44]:
#Citizen's Bank Park (Phillies Stadium)
latitude = 39.9057
longitude = -75.1665
date = '2025-06-30'


hourly = data['hourly']
print(hourly.keys())

dict_keys(['time', 'wind_direction_10m', 'wind_speed_10m'])


In [49]:
print(data.keys())

dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly'])


In [48]:
# Preview the first 5 hourly entries
for i in range(5):
    print(f"Time: {hourly['time'][i]}, Direction: {hourly['wind_direction_10m'][i]}°, Speed: {hourly['wind_speed_10m'][i]}")


Time: 2025-06-30T00:00, Direction: 193°, Speed: 3.3
Time: 2025-06-30T01:00, Direction: 183°, Speed: 6.1
Time: 2025-06-30T02:00, Direction: 135°, Speed: 1.0
Time: 2025-06-30T03:00, Direction: 347°, Speed: 4.8
Time: 2025-06-30T04:00, Direction: 139°, Speed: 3.3
